In [1]:
import numpy as np
import pandas as pd

# Unit Tests

## Overview and Principles
Testing is the process by which you exercise your code to determine if it performs as expected. The code you are testing is referred to as the **code under test**. 

There are two parts to writing tests.
1. invoking the code under test so that it is exercised in a particular way;
1. evaluating the results of executing code under test to determine if it behaved as expected.

The collection of tests performed are referred to as the **test cases**. The fraction of the code under test that is executed as a result of running the test cases is referred to as **test coverage**.

For dynamical languages such as Python, it's extremely important to have a high test coverage. In fact, you should try to get 100% coverage. This is because little checking is done when the source code is read by the Python interpreter. For example, the code under test might contain a line that has a function that is undefined. This would not be detected until that line of code is executed.

Test cases can be of several types. Below are listed some common classifications of test cases.
- *Smoke test*. This is an invocation of the code under test to see if there is an unexpected exception. It's useful as a starting point, but this doesn't tell you anything about the correctness of the results of a computation.
- *One-shot test*. In this case, you call the code under test with arguments for which you know the expected result.
- *Edge test*. The code under test is invoked with arguments that should cause an exception, and you evaluate if the expected exception occurrs.
- *Pattern test* - Based on your knowledge of the *calculation* (not implementation) of the code under test, you construct a suite of test cases for which the results are known or there are known patterns in these results that are used to evaluate the results returned.

Another principle of testing is to limit what is done in a single test case. Generally, a test case should focus on one use of one function. Sometimes, this is a challenge since the function being tested may call other functions that you are testing. This means that bugs in the called functions may cause failures in the tests of the calling functions. Often, you sort this out by knowing the structure of the code and focusing first on failures in lower level tests. In other situations, you may use more advanced techniques called *mocking*. A discussion of mocking is beyond the scope of this lecture.

## Test-driven development

A best practice is to develop your tests while you are developing your code. Indeed, one school of thought in software engineering, called **test-driven development**, advocates that you write the tests *before* you implement the code under test so that the test cases become a kind of specification for what the code under test should do.

**This is how you should approach development going forward in this course.** Write your tests first.  They all fail.  Write the code for the functions to make the tests pass.

## Examples of Test Cases
This section presents examples of test cases. The code under test is the calculation of entropy.

### Entropy of a set of probabilities
$$
H = -\sum_i p_i \log(p_i)
$$
where $\sum_i p_i = 1$.

In [25]:
# Code Under Test
def entropy(ps):
    if any([(p < 0.0) or (p > 1.0) for p in ps]):
        raise ValueError("At least one input is out of range [0...1]")
    else:
        pass
    if not np.isclose(1, np.sum(ps), atol=1e-08):
        raise ValueError("The list of input probabilities does not sum to 1")
    else:
        pass
    items = ps * np.log2(ps)
    new_items = []
    for item in items:
        if np.isnan(item):
            new_items.append(0)
        else:
            new_items.append(item)
    return np.abs(-np.sum(new_items))

In [26]:
ps = [.8, .2]
#ps = [ 1.00000001, 0]
[(p < 0.0) or (p > 1.0) for p in ps]

[False, False]

### Smoke test 

Does the function run when we call it or does it explode in flames and release the magic smoke.

In [27]:
# Smoke test
entropy([0.5, 0.5])

1.0

### One shot test

We know from previous discussions that when we have 4 states and they are all equally likely, the number of bits required should be 2.

In [28]:
entropy([.25, .25, .25, .25])

2.0

Another example is that the entropy of a random variable when there is only one possible outcome is 0, therefore:

In [29]:
entropy([1])

0.0

Suppose that all of the probability of a distribution is at one point. An example of this is a coin with two heads. Whenever you flip it, you always get heads. That is, the probability of a head is 1.

What is the entropy of such a distribution? From the calculation above, we see that the entropy should be $log(1)$, which is 0. This means that we have a test case where we know the result!

In [30]:
entries = [
    [0, [1]],
]

for entry in entries:
    ans = entry[0]
    prob = entry[1]
    if not np.isclose(entropy(prob), ans):
        print("Test failed!")
print ("Test completed!")

Test completed!


**NEAT!** We can use this structure to do a bunch of these types all at once, e.g.

In [31]:
entries = [
    [0, [1]],
    [2, [.25, .25, .25, .25]]
]

for idx, entry in enumerate(entries):
    ans = entry[0]
    prob = entry[1]
    if not np.isclose(entropy(prob), ans):
        print(f"Test {idx+1} failed")
    else:
        print(f"Test {idx+1} passed")
print ("Test completed!")

Test 1 passed
Test 2 passed
Test completed!


**Question**: What is an example of another one-shot test? (Hint: You need to know the expected result.)

### Edge tests

One edge test of interest is to provide an input that is *not* a distribution in that probabilities don't sum to 1.  These should generate an exception of type ValueError

In [32]:
entropy([.9, .9])

ValueError: The list of input probabilities does not sum to 1

Another edge test is when we pass a probability that is out of range.

In [33]:
entropy([-0.5])

ValueError: At least one input is out of range [0...1]

#### Important note for edge tests that raise exceptions!

You often have to write your tests using `try` and `except` blocks being sure to catch the correct Exception type, e.g.

In [34]:
def test_entropy_parameter_checking():
    # first, let's try not summing to 1
    try:
        entropy([.9, .9])
    except (ValueError) as err:
        print("Test of probability inputs not summing to 1 passed")
    try:
        entropy([-0.5])
    except (ValueError) as err:
        print("Test of probability input ranges passed")

In [35]:
test_entropy_parameter_checking()

Test of probability inputs not summing to 1 passed
Test of probability input ranges passed


Now let's consider a pattern test. Examining the structure of the calculation of $H$, we consider a situation in which there are $n$ equal probabilities. That is, $p_i = \frac{1}{n}$.
$$
H = -\sum_{i=1}^{n} p_i \log(p_i) 
= -\sum_{i=1}^{n} \frac{1}{n} \log(\frac{1}{n}) 
= n (-\frac{1}{n} \log(\frac{1}{n}) )
= -\log(\frac{1}{n})
$$
For example, entropy([0.5, 0.5]) should be $-log(0.5)$.

In [36]:
# Pattern test
def test_equal_probabilities(n):
    prob = 1.0/n
    ps = np.repeat(prob , n)
    if np.isclose(entropy(ps), -np.log2(prob)):
        print(f"Test passed for n = {n}")
    else:
        import pdb; pdb.set_trace()
        print (f"Test failed for n = {n}")
        
        
# Run a test
test_equal_probabilities(100000)

Test passed for n = 100000


You see that there are many, many cases to test. So far, we've been writing special codes for each test case. We can do better.

## Testing Data Producing Codes
Much of your python (or R) codes will be creating and/or transforming dataframes. A dataframe is structured like a table with:

- Columns that have values of the same type
- Rows that have a value for each column
- An index that uniquely identifies a row.

In [37]:
def makeProbabilityMatrix(column_names, nrows):
    """
    Makes a dataframe with the specified column names such that each
    cell is a value in [0, 1] and columns sum to 1.
    :param list-str column_names: names of the columns
    :param int nrows: number of rows
    """
    df = pd.DataFrame(np.random.uniform(0, 1, (nrows, len(column_names))))
    df.columns = column_names
    for column in df.columns:
        df[column] = df[column]/df[column].sum()
    return df
                      

#### Smoke test

In [38]:
makeProbabilityMatrix(['a', 'b'], 3)

,a,b
0,0.041695,0.205093
1,0.561708,0.776799
2,0.396597,0.018107


In [39]:
# Test 2: Check columns
COLUMNS = ['a', 'b']
df = makeProbabilityMatrix(COLUMNS, 3)
set(COLUMNS) == set(df.columns)

True

### Exercise
Write a function that tests the following:
- The returned dataframe has the expected columns
- The returned dataframe has the expected rows
- Values in columns are of the correct type and range
- Values in column sum to 1

## Pytest Infrastructure

There are several reasons to use a test infrastructure:
- If you have many test cases (which you should!), the test infrastructure will save you from writing a lot of code.
- The infrastructure provides a uniform way to report test results, and to handle test failures.
- A test infrastructure can tell you about coverage so you know what tests to add.

We'll be using the `Pytest` framework. This is a separate Python package. Using this infrastructure, requires the following:
1. import the pytest module
1. write methods that run the code to be tested and check the outcomes.

You indicate that a method is to be run as a test by having the method name begin with "test".

Second, the "test methods" should communicate with the infrastructure the results of evaluating output from the code under test. This is done by using `assert` statements. For example, `np.testing.assert_equal` takes two arguments. If these are objects for which `==` returns `True`, then the test passes. Otherwise, the test fails.

In [8]:
import pytest
import ipytest
import numpy as np
ipytest.autoconfig()

In [40]:
%%ipytest

# Define test functions

def test_success():
    np.testing.assert_equal(1, 1)

def test_success1():
    assert 1 == 1

def test_failure():
    assert 1 ==2


..F                                                                                          [100%]
============================================= FAILURES =============================================
___________________________________________ test_failure ___________________________________________

    def test_failure():
>       assert 1 ==2
E       assert 1 == 2

/var/folders/x0/30gk1h9n3fqfnt97pwz4bw380000gn/T/ipykernel_23627/1613237084.py:10: AssertionError
===================================== short test summary info ======================================
FAILED t_2cf8ebce99214601997bab7000bcce29.py::test_failure - assert 1 == 2
1 failed, 2 passed in 0.74s


In [ ]:
# Function the handles test loading

**Code for homework or your work should use test files.** In this lesson, we'll show how to write test codes in a Jupyter notebook. This is done for pedidogical reasons. It is **NOT** not something you should do in practice, except as an intermediate exploratory approach. 

As expected, the first test passes, but the second test fails.

### Exercise
- Rewrite the above one-shot test for entropy using the pytest infrastructure.

In [17]:
# Implementating a pattern test. Use functions in the test.
import pytest
        
def test_equal_probability():
    def test(count):
        """
        Invokes the entropy function for a number of values equal to count
        that have the same probability.
        :param int count:
        """
        raise RuntimeError ("Not implemented.")
    #
    test(2)
    test(20)
    test(200)

test_equal_probability()

RuntimeError: Not implemented.

In [ ]:
import pytest

# Define test functions for the entropy function

## Testing For Exceptions

Edge test cases often involves handling exceptions. One approach is to code this directly.

In [44]:
%%ipytest

import pytest

# Define tests to run
        
def test_invalid_probability():
    try:
        entropy([0.1, 0.5])
        assert False
    except ValueError:
        assert True

.                                                                                            [100%]
1 passed in 0.01s


`pytest` provides help with testing exceptions.

In [41]:
%%ipytest

import pytest

# Define tests

def test_invalid_probability():
    with pytest.raises(ValueError):
        entropy([0.1, 0.5])
        


.                                                                                            [100%]
1 passed in 0.87s


## Test Files
Although I presented the elements of `pytest` in a notebook. **your tests should be in a file**. If the name of module with the code under test is `foo.py`, then the name of the test file should be `test_foo.py`.

The structure of the test file will be very similar to cells above. You will import `pytest`. You must also import the module with the code under test. Take a look at `test_prime.py` in this directory to see an example.

## Discussion
**Question**: What tests would you write for a plotting function?

## Test Driven Development
Start by writing the tests. Then write the code.

We illustrate this by considering a function geomean that takes a list of numbers as input and produces the geometric mean on output.

In [45]:
%%ipytest
import pytest

# Define tests
        
def test_oneshot():
    assert geomean([1,1]) == 1
    
def test_oneshot2():
    assert geomean([3, 3, 3]) == 3
        


FF                                                                                           [100%]
============================================= FAILURES =============================================
___________________________________________ test_oneshot ___________________________________________

    def test_oneshot():
>       assert geomean([1,1]) == 1
E       NameError: name 'geomean' is not defined

/var/folders/x0/30gk1h9n3fqfnt97pwz4bw380000gn/T/ipykernel_23627/481383766.py:6: NameError
__________________________________________ test_oneshot2 ___________________________________________

    def test_oneshot2():
>       assert geomean([3, 3, 3]) == 3
E       NameError: name 'geomean' is not defined

/var/folders/x0/30gk1h9n3fqfnt97pwz4bw380000gn/T/ipykernel_23627/481383766.py:9: NameError
===================================== short test summary info ======================================
FAILED t_2cf8ebce99214601997bab7000bcce29.py::test_oneshot - NameError: name 'geomean' is

In [ ]:
#def geomean(argument?):
#    return ?

## Other infrastructures
- pytest
- nose
- Use binary functions that being with "test"

## References

https://www.youtube.com/watch?v=GEqM9uJi64Q (Pydata 2015)
https://www.youtube.com/watch?v=yACtdj1_IxE (Pycon 2017)

The first talk mentions some packages:
engarde - https://github.com/TomAugspurger/engarde
Hypothesis - https://hypothesis.readthedocs.io/en/latest/
Feature Forge - https://github.com/machinalis/featureforge


Detlef Nauck talk: 
http://ukkdd.org.uk/2017/info/talks/nauck.pdf
He also had a list of R tools but I could not find the slides form the talk I saw.

Test Driven Data Analysis:
https://www.youtube.com/watch?v=TGwZnZYg0jw

Profiling for Pandas:
https://github.com/pandas-profiling/pandas-profiling